In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 13
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
# 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df, n):
    print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
    summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
    summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

    summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
    summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산

    for i in range(n):
        summary[f"{i+1}번째 값"] = df.iloc[i].values

    return summary                                                     # 요약 테이블 반환

In [ ]:
# 2. pandas로 데이터 불러오기
import pandas as pd

# ▶ Feature 파일 불러오기
df_features = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/unique_1/train_1_회원정보_모델링용.csv')

In [ ]:
# ▶ Segment 파일 불러오기 (ID + Segment)
df_segment = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/unique_1/Segment_병합.csv')

# ▶ Segment만 순서대로 붙이기 (ID 순서 동일하다는 전제)
df_features['Segment'] = df_segment['Segment'].values

# ▶ 확인
display(resumetable(df_features,3))
df_features[['ID', 'Segment']]

데이터셋 크기: (2400000, 66)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,남녀구분코드,int64,0,2,2,1,1
3,회원여부_이용가능,int64,0,2,1,1,1
4,회원여부_이용가능_CA,int64,0,2,1,1,1
...,...,...,...,...,...,...,...
61,Life_Stage,object,0,7,자녀성장(2),자녀성장(1),자녀출산기
62,최종카드발급경과월,int64,0,68,22,18,20
63,연령_num,int64,0,6,45,35,35
64,연회비발생카드여부_B0M,int64,0,2,0,0,0


,ID,Segment
0,TRAIN_000000,D
1,TRAIN_000001,E
2,TRAIN_000002,C
3,TRAIN_000003,D
4,TRAIN_000004,E
...,...,...
2399995,TRAIN_399995,E
2399996,TRAIN_399996,D
2399997,TRAIN_399997,C
2399998,TRAIN_399998,E


In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1. Segment를 숫자로 변환
le = LabelEncoder()
df_features['Segment_encoded'] = le.fit_transform(df_features['Segment'])

# 2. 숫자형 컬럼만 선택
numeric_cols = df_features.select_dtypes(include=['number']).columns.drop('Segment_encoded', errors='ignore')

# 3. 상관계수 계산
correlations = df_features[numeric_cols].corrwith(df_features['Segment_encoded'])

# ✅ 절댓값 적용 여부 → 아래 한 줄 주석처리로 토글
correlations = correlations.abs()

# 4. 정렬
correlations_sorted = correlations.sort_values(ascending=False)

# 5. 출력 형식
pd.set_option('display.float_format', lambda x: f'{x:.6f}')
pd.set_option('display.max_rows', None)

# 6. 출력
display(correlations_sorted)

,0
이용금액_R3M_신용체크,0.622827
이용금액_R3M_신용,0.589032
_1순위카드이용금액,0.573870
_2순위카드이용금액,0.400225
이용카드수_신용체크,0.397713
_1순위카드이용건수,0.383609
이용카드수_신용,0.376918
_2순위카드이용건수,0.376494
이용가능카드수_신용,0.353014
이용가능카드수_신용체크,0.352211


In [ ]:
# 7. 상관계수 0.35 이상 컬럼만 선택
selected_cols = correlations_sorted[correlations_sorted >= 0.35].index.tolist()

# 8. 해당 컬럼만 추출
X_selected = df_features[selected_cols]

# 9. 확인
print("선택된 컬럼 수:", len(selected_cols))
display(correlations[selected_cols])

선택된 컬럼 수: 10


,0
이용금액_R3M_신용체크,0.622827
이용금액_R3M_신용,0.589032
_1순위카드이용금액,0.573870
_2순위카드이용금액,0.400225
이용카드수_신용체크,0.397713
_1순위카드이용건수,0.383609
이용카드수_신용,0.376918
_2순위카드이용건수,0.376494
이용가능카드수_신용,0.353014
이용가능카드수_신용체크,0.352211


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# vif를 계산하기 위한 상수항을 추가한다.
vif_X = sm.add_constant(X_selected)

# vif를 계산한다.
vif = pd.DataFrame()
vif['변수'] = vif_X.columns
vif['VIF'] = [variance_inflation_factor(vif_X.values, i) for i in range(vif_X.shape[1])]
vif

,변수,VIF
0,const,5.313550
1,이용금액_R3M_신용체크,14.107237
2,이용금액_R3M_신용,84.260303
3,_1순위카드이용금액,41.933094
4,_2순위카드이용금액,27.265139
5,이용카드수_신용체크,13.225589
6,_1순위카드이용건수,3.112086
7,이용카드수_신용,15.804013
8,_2순위카드이용건수,14.485353
9,이용가능카드수_신용,6.093288


In [25]:
# vif 높은 컬럼 하나씩 제거
vif_X = X_selected.drop([
    '이용금액_R3M_신용',
    '이용카드수_신용',
    '_2순위카드이용금액'
    ], axis=1)

# vif를 계산하기 위해 상수항을 추가한다.
vif_X2 = sm.add_constant(vif_X)

# vif를 계산한다.
vif = pd.DataFrame()
vif['변수'] = vif_X2.columns
vif['VIF'] = [variance_inflation_factor(vif_X2.values, i) for i in range(vif_X2.shape[1])]
vif

,변수,VIF
0,const,5.022806
1,이용금액_R3M_신용체크,9.312666
2,_1순위카드이용금액,7.786934
3,이용카드수_신용체크,3.743859
4,_1순위카드이용건수,2.658381
5,_2순위카드이용건수,3.322235
6,이용가능카드수_신용,3.852371
7,이용가능카드수_신용체크,3.012366
